In [1]:
import cv2
import numpy as np
import helpers


In [2]:
image = cv2.imread('puzz1.jpg',0)
outerBox = np.ndarray(image.shape)

<a href='https://www.tutorialkart.com/opencv/python/opencv-python-gaussian-image-smoothing/'>Gaussian Blur</a>

In [3]:
image = cv2.GaussianBlur(image,(11,11),0)

<a href="https://pyimagesearch.com/2021/05/12/adaptive-thresholding-with-opencv-cv2-adaptivethreshold/">Adaptive Thresholding</a>

In [4]:
outerBox = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 5, 2)

In [8]:
cv2.imshow('before bitwise',outerBox)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
outerBox = cv2.bitwise_not(outerBox)

<a href='https://docs.opencv.org/3.4/db/df6/tutorial_erosion_dilatation.html'>Dilation</a>

In [15]:
outerBox = cv2.dilate(outerBox, cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3)),iterations=4)

In [16]:
cv2.imshow('here', outerBox)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [28]:
outerBox.shape

(1167, 1125)

In [46]:
contours, heirarchy = cv2.findContours(outerBox, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [47]:
canvas = np.ndarray(image.shape)

canvas = cv2.drawContours(canvas,contours,0,100,4)

In [63]:
contours[0][:,0,:]

array([[  14,   19],
       [  10,   23],
       [  10, 1119],
       [  14, 1123],
       [1110, 1123],
       [1114, 1119],
       [1114,   25],
       [1115,   24],
       [1110,   19]], dtype=int32)

In [64]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4,random_state=0).fit(contours[0][:,0,:])

In [66]:
corners = kmeans.cluster_centers_